In [2]:
import os
import json
import pandas as pd
from collections import defaultdict, Counter


### Bomb Risk Game

1 safe, 0 bomb

#### Human Data

In [7]:
# Data: 
#   Round
#       Number of bombs
#           decision maker?
#           bomb ordering
#           choice
df = pd.read_csv('records/bomb_risk.csv')
df = df[df['Role'] == 'player']
df = df[df['gameType'] == 'bomb_risk']
df = df[df['Round'] <= 3]
df.sort_values(by=['UserID', 'Round'])

,UserID,Role,Round,move,gameType,roundResult
2099,0002b694-1f33-45de-afcc-e1c9f9dffe78,player,1,69.0,bomb_risk,SAFE
34634,0007d903-2df5-4833-b928-35361ef46432,player,1,42.0,bomb_risk,SAFE
63027,0008c671-a0a3-4fbe-988b-ef28d3811853,player,1,-1.0,bomb_risk,BOMB
61108,000990be-e55e-460f-aac2-ae114917d4aa,player,1,90.0,bomb_risk,BOMB
36456,000aaaa4-afac-4cd7-97dc-fd2232375dea,player,1,20.0,bomb_risk,SAFE
...,...,...,...,...,...,...
19065,fff8ea6b-dbe1-4b1e-ba29-5529349439e5,player,1,39.0,bomb_risk,SAFE
19066,fff8ea6b-dbe1-4b1e-ba29-5529349439e5,player,2,39.0,bomb_risk,BOMB
19067,fff8ea6b-dbe1-4b1e-ba29-5529349439e5,player,3,39.0,bomb_risk,BOMB
4318,fffd236c-e632-4300-992f-5e55942db076,player,1,50.0,bomb_risk,BOMB


In [11]:

round_choices = [] 
for i in range(0, df['Round'].max()):
    round_choices.append({'Human': defaultdict(list), 'ChatGPT-4': defaultdict(list), 'ChatGPT-3': defaultdict(list),})
prev_user = None
# prev_move = None
bad_user = False
round = 1
prefix = '0'
exit = False

for _, row in df.iterrows():
    # if user is invalid, don't use data
    if bad_user: continue
    # Move onto new user
    if row['UserID'] != prev_user:
        prev_user = row['UserID']
        round = 1
        prefix = '0'
        bad_user = False
    if row['Round'] != round:
        continue
    
    move = row['move']
    if move < 0 or move > 100:
        bad_users = True
        continue
    round_choices[row['Round'] - 1]['Human'][prefix].append(move)

    prefix += '1' if row['roundResult'] == 'SAFE' else '0'
    round = round + 1
print(round_choices)

[{'Human': defaultdict(<class 'list'>, {'0': [33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 70.0, 80.0, 49.0, 44.0, 50.0, 20.0, 25.0, 44.0, 18.0, 25.0, 50.0, 42.0, 50.0, 50.0, 11.0, 50.0, 49.0, 35.0, 50.0, 34.0, 50.0, 77.0, 99.0, 51.0, 40.0, 30.0, 40.0, 50.0, 20.0, 55.0, 100.0,

#### Model Data

In [12]:
# Round
#   Decision Maker
#       Prefix
#           choice 

prefix_to_choices_model = defaultdict(lambda : defaultdict(list))
for model in ['ChatGPT-4', 'ChatGPT-3']:
    if model == 'ChatGPT-4':
        file_names = [
            'bomb_gpt4_2023_05_15-12_13_51_AM.json'
        ]
    elif model == 'ChatGPT-3':
        file_names = [
            'bomb_turbo_2023_05_14-10_45_50_PM.json'
        ]

    choices = []
    scenarios = []
    for file_name in file_names:
        with open(os.path.join('records', file_name), 'r') as f:
            records = json.load(f)
            choices += records['choices']
            scenarios += records['scenarios']

    assert len(scenarios) == len(choices)
    print('loaded %i valid records' % len(scenarios))

    # prefix_to_choice = defaultdict(list)
    # prefix_to_result = defaultdict(list)
    # prefix_to_pattern = defaultdict(Counter)
    wrong_sum = 0
    for scenarios_tmp, choices_tmp in zip(scenarios, choices):

        result = 0
        for i, scenario in enumerate(scenarios_tmp):
            prefix = tuple(scenarios_tmp[:i])
            prefix = ''.join([str(x) for x in prefix])
            choice = choices_tmp[i]
            
            round_choices[len(prefix) - 1][model][prefix].append(choice)
            # prefix_to_choice[prefix].append(choice)
            # prefix_to_pattern[prefix][tuple(choices_tmp[:-1])] += 1

            prefix = tuple(scenarios_tmp[:i+1])
            # if scenario == 1:
            #     result += choice
            # prefix_to_result[prefix].append(result)

    # print('# of wrong sum:', wrong_sum)
    # print('# of correct sum:', len(scenarios) - wrong_sum)

    # prefix_to_choices_model[model] = prefix_to_choice
    

loaded 80 valid records
loaded 80 valid records


#### Export

In [13]:
with open('../frontend/src/data.json', 'w') as file:
    json.dump(round_choices, file)